# Práctica 1: Analitica Social y de la Web

## Tareas: 
1. Obtener una lista de palabras en español y otra en inglés con el resultado de
limpiar con los pasos anteriores el párrafo de ejemplo

2. Obtener una lista de palabras con el resultado de limpiar con los pasos
anteriores el texto de la web www.alotuyo.com

3. Obtener una lista de palabras con el resultado de limpiar con los pasos
anteriores las 10 primeras reseñas de la película “Titanic” en www.imdb.com

### Entrega de ficheros .ipynb por correo electrónico: miércoles 7 de febrero.

In [1]:
#Libraries
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from nltk import word_tokenize
from unidecode import unidecode
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import time
import re 
import nltk
import requests

In [2]:
# * Archivos
# nltk.download('stopwords')
spanish_dict = 'diccionarios/spanish-wordlist.txt'
english_dict = 'diccionarios/english-wordlist.txt'

with open('diccionarios/english-wordlist.txt','r') as file:
    english_dict_words = set(file.read().splitlines())

with open('diccionarios/spanish-wordlist.txt','r') as file: 
    spanish_dict_words = set(file.read().splitlines())

### Pasos de limpieza (Cuidado con el orden)
* Eliminar espacios en blanco
* Eliminar puntuación, tags de HTML y URLs
* Estandarizar palabras (eliminar letras o símbolos repetidos)
* Eliminar palabras concatenadas
* Pasar a minúsculas
* Eliminar palabras comunes
* Tokenizar las palabras resultantes

In [3]:
def clean_text(text_input, language):
    # Check if the language is supported
    if language not in ['english', 'spanish']:
        raise ValueError('Error: Solo se puede seleccionar Español e Inglés.')

    # Escogemos el diccionario basado en la lengua
    dict_words = english_dict_words if language == 'english' else spanish_dict_words

    # 1. Convertimos el texto a minusculas
    pure_text = re.sub(r'[^\w\s]', '', text_input.lower())
    
    # 2. Quitamos las stop words.
    pure_text = ' '.join([word for word in pure_text.split() if word not in stopwords.words(language)])
    
    # 3. Filtrar palabras no encontradas en el diccionario. 
    filtered_words = [word for word in word_tokenize(pure_text) if word in dict_words]
    incorrect_words = [word for word in word_tokenize(pure_text) if word not in dict_words]
    
    # Opcional: Para manejar las palabras juntas es dificil separarlas.

    return filtered_words, incorrect_words


#### Ejemplo Texto (Ingles): 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its
contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not
meant that we should voyage far. The sciences, each straining in its own direction, have hitherto
harmed us little; but some day the piecing together of dissociated knowledge will open up such
terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the
revelation or flee from the deadly light into the peace and safety of a new dark age.
Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and
human race form transient incidents. They have hinted at strange survivals in terms which would
freeze the blood if not masked by a bland optimism. But it is not from them that there came the
single glimpse of forbidden aeons which chills me when I think of it and maddens me when I dream
of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together
of separated things—in this case an old newspaper item and the notes of a dead professor. I hope
that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a
link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part
he knew, and that he would have destroyed his notes had not sudden death seized him.

In [4]:
sample_text = """The most merciful thing in the world, I think, is the inability of the human mind to correlate all its
contents. We live on a placid island of ignorance in the midst of black seas of infinity, and it was not
meant that we should voyage far. The sciences, each straining in its own direction, have hitherto
harmed us little; but some day the piecing together of dissociated knowledge will open up such
terrifying vistas of reality, and of our frightful position therein, that we shall either go mad from the
revelation or flee from the deadly light into the peace and safety of a new dark age.
Theosophists have guessed at the awesome grandeur of the cosmic cycle wherein our world and
human race form transient incidents. They have hinted at strange survivals in terms which would
freeze the blood if not masked by a bland optimism. But it is not from them that there came the
single glimpse of forbidden aeons which chills me when I think of it and maddens me when I dream
of it. That glimpse, like all dread glimpses of truth, flashed out from an accidental piecing together
of separated things—in this case an old newspaper item and the notes of a dead professor. I hope
that no one else will accomplish this piecing out; certainly, if I live, I shall never knowingly supply a
link in so hideous a chain. I think that the professor, too, intended to keep silent regarding the part
he knew, and that he would have destroyed his notes had not sudden death seized him."""

pure_text, incorrect_words = clean_text(sample_text,'english')
print(f"Corrected text: f{pure_text}")
print(f"Incorrect_words: f{incorrect_words}")

Corrected text: f['merciful', 'thing', 'world', 'think', 'inability', 'human', 'mind', 'correlate', 'contents', 'live', 'placid', 'island', 'ignorance', 'midst', 'black', 'seas', 'infinity', 'meant', 'voyage', 'far', 'sciences', 'straining', 'direction', 'hitherto', 'harmed', 'us', 'little', 'day', 'piecing', 'together', 'dissociated', 'knowledge', 'open', 'terrifying', 'vistas', 'reality', 'frightful', 'position', 'therein', 'shall', 'either', 'go', 'mad', 'revelation', 'flee', 'deadly', 'light', 'peace', 'safety', 'new', 'dark', 'age', 'theosophists', 'guessed', 'awesome', 'grandeur', 'cosmic', 'cycle', 'wherein', 'world', 'human', 'race', 'form', 'transient', 'incidents', 'hinted', 'strange', 'survivals', 'terms', 'would', 'freeze', 'blood', 'masked', 'bland', 'optimism', 'came', 'single', 'glimpse', 'forbidden', 'aeons', 'chills', 'think', 'maddens', 'dream', 'glimpse', 'like', 'dread', 'glimpses', 'truth', 'flashed', 'accidental', 'piecing', 'together', 'separated', 'case', 'old',

## Traducimos del inglés al español

In [5]:
spanish_text = GoogleTranslator(source='auto', target='es').translate(sample_text)
sample_text_es = spanish_text
print(sample_text_es)

Creo que lo más misericordioso del mundo es la incapacidad de la mente humana para correlacionar todas sus
contenido. Vivimos en una plácida isla de ignorancia en medio de mares negros del infinito, y no fue
significaba que debíamos viajar lejos. Las ciencias, cada una de las cuales se esfuerza en su propia dirección, hasta ahora han
nos hizo daño poco; pero algún día la unión del conocimiento disociado abrirá tales
perspectivas aterradoras de la realidad y de nuestra espantosa posición en ella, que nos volveremos locos por la
revelación o huir de la luz mortal hacia la paz y la seguridad de una nueva era oscura.
Los teósofos han adivinado la asombrosa grandeza del ciclo cósmico en el que nuestro mundo y
La raza humana forma incidentes transitorios. Han insinuado extrañas supervivencias en términos que
congelar la sangre si no está enmascarado por un suave optimismo. Pero no es de ellos de donde vino el
Un solo vistazo de eones prohibidos que me estremece cuando pienso en ello y me enl

In [6]:
pure_text_es, incorrect_words_es = clean_text(sample_text_es,'spanish')
print(pure_text_es)

['creo', 'misericordioso', 'mundo', 'incapacidad', 'mente', 'humana', 'todas', 'contenido', 'vivimos', 'isla', 'ignorancia', 'medio', 'mares', 'negros', 'infinito', 'significaba', 'viajar', 'lejos', 'ciencias', 'cada', 'cuales', 'esfuerza', 'propia', 'ahora', 'hizo', 'daño', 'conocimiento', 'disociado', 'tales', 'perspectivas', 'aterradoras', 'realidad', 'espantosa', 'volveremos', 'locos', 'huir', 'luz', 'mortal', 'hacia', 'paz', 'seguridad', 'nueva', 'oscura', 'adivinado', 'asombrosa', 'grandeza', 'ciclo', 'mundo', 'raza', 'humana', 'forma', 'incidentes', 'transitorios', 'insinuado', 'extrañas', 'supervivencias', 'congelar', 'sangre', 'si', 'enmascarado', 'suave', 'optimismo', 'vino', 'solo', 'vistazo', 'eones', 'prohibidos', 'estremece', 'pienso', 'ello', 'enloquece', 'sueño', 'ello', 'destello', 'temibles', 'destellos', 'verdad', 'accidental', 'cosas', 'separadas', 'caso', 'viejo', 'notas', 'profesor', 'muerto', 'espero', 'nadie', 'ciertamente', 'si', 'vivo', 'nunca', 'tan', 'espant

---

# Parte II: Limpieza de texto de la pagina web www.alotuyo.com

In [7]:
url = "https://alotuyo.com/"

def scrape_site(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return ' '.join([p.text for p in soup.find_all('p')])

website_text = scrape_site(url)
print(website_text)



						🔥 Alotuyo
					
 Forma parte de la comunidad que no se conforma con el camino establecido Nuevos recursos gratuitos, cada semana Revisa tu bandeja de entrada o la carpeta de spam para confirmar tu suscripción.
            Conecta, viaja, aprende, comparte y crece junto a +200 piratas por todo el mundo Con tu suscripción recibirás +50h de contenido (vídeo y podcast)Cada mes desbloqueamos 2 nuevas sesiones.  Esta es una de las herramientas a las que tendrás acceso cuando entres. Descubre los mejores lugares para comer, trabajar, disfrutar y dormir en todo el mundo.  Lo digital está bien, pero a nosotros nos gusta vernos en el mundo real. Conoce a la comunidad en las diferentes quedadas y co-livings que organizamos  Paga una vez y accede a todo el contenido, recursos, comunidad, recomendaciones y contactos A ver si con esto te las resolvemos. Y si hay algo que no te queda claro, escríbenos un whatsap 🙂 Comunidad creada por y para nómadas digitales hispanohablantes. Conectamos a pe

In [8]:
pure_website_text, incorrect_website_words = clean_text(website_text,'spanish')
print(pure_website_text)
print(len(pure_website_text))
print(incorrect_website_words)
print(len(incorrect_website_words))
print(f"Error Rate: {len(incorrect_website_words)/(len(incorrect_website_words) + len(pure_website_text))}")

['forma', 'parte', 'comunidad', 'conforma', 'camino', 'establecido', 'nuevos', 'recursos', 'gratuitos', 'cada', 'semana', 'revisa', 'bandeja', 'entrada', 'carpeta', 'confirmar', 'conecta', 'viaja', 'aprende', 'comparte', 'crece', 'junto', 'piratas', 'mundo', 'contenido', 'mes', 'desbloqueamos', 'nuevas', 'sesiones', 'herramientas', 'acceso', 'entres', 'descubre', 'mejores', 'lugares', 'comer', 'trabajar', 'disfrutar', 'dormir', 'mundo', 'digital', 'bien', 'gusta', 'mundo', 'real', 'conoce', 'comunidad', 'diferentes', 'quedadas', 'organizamos', 'paga', 'vez', 'accede', 'contenido', 'recursos', 'comunidad', 'recomendaciones', 'contactos', 'ver', 'si', 'resolvemos', 'si', 'queda', 'claro', 'comunidad', 'creada', 'digitales', 'conectamos', 'personas', 'gusta', 'trabajar', 'emprender', 'remoto', 'cualquier', 'lugar', 'mundo', 'ofrecemos', 'contenido', 'negocios', 'digitales', 'fiscalidad', 'internacional', 'recomendaciones', 'viajes', 'organizar', 'eventos', 'quedadas', 'colaborar', 'comuni

---
# Parte 3: Scrapping de las 10 primeras criticas de Titanic en IMDb

In [9]:
REVIEW_NUMBER = 10

def extract_imdb_reviews_first_page(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    reviews = []
    for item in soup.find_all('div', class_='review-container'):
        reviews.append(item.text.strip())
    
    return reviews

In [10]:
titanic_reviews_first_page_list = extract_imdb_reviews_first_page('tt0120338')

def clean_reviews_english(review_list):
    cleaned_reviews = []
    errors = []

    for review in review_list:
        clean_review, error = clean_text(review, 'english')
        cleaned_reviews.append(clean_review)
        errors.append(error)
        print(clean_review)
        print(f'\n{error}')
        print('---')

    return cleaned_reviews, errors


titanic_reviews_first_page_keywords = clean_reviews_english(titanic_reviews_first_page_list)
print(titanic_reviews_first_page_keywords)

['many', 'times', 'watch', 'movie', 'still', 'masterpiece', 'september', 'watched', 'titanic', 'many', 'times', 'know', 'watch', 'still', 'cry', 'laugh', 'smile', 'feel', 'story', 'flows', 'tension', 'throughout', 'movie', 'two', 'actors', 'acting', 'chemistry', 'need', 'applaud', 'sinking', 'ship', 'realistically', 'filmed', 'heart', 'go', 'perfect', 'fit', 'jack', 'roses', 'love', 'story', 'timeless', 'well', 'movies', 'factors', 'fully', 'qualified', 'say', 'masterpiece', 'found', 'helpful', 'review', 'helpful', 'sign', 'vote']

['1010', 'tayloryee942', '2020', 'dont', 'everytime', '210', '227', 'permalink']
---
['standing', 'ovation', 'stars', 'march', 'stage', 'curtains', 'open', 'since', 'advent', 'film', 'ever', 'enthralled', 'emotionally', 'invested', 'movie', 'titanic', 'knew', 'going', 'trailer', 'going', 'good', 'idea', 'ride', 'lights', 'dimmed', 'theater', 'night', 'never', 'understood', 'still', 'understand', 'negative', 'backlash', 'come', 'epic', 'film', 'epic', 'every'

### Bonus - Extraer 40 reseñas de la pagina web - Selenium

In [11]:
REVIEW_NUMBER = 40

def extract_imdb_reviews_all(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews'
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in background
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)

    reviews = []

    try:
        while len(reviews) < REVIEW_NUMBER:
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            for item in soup.find_all('div', class_='text show-more__control'):
                reviews.append(item.text.strip())
                if len(reviews) >= REVIEW_NUMBER:
                    break

            # Attempt to click the "Load More" button to load more reviews
            try:
                load_more_button = driver.find_element(By.CLASS_NAME, 'ipl-load-more__button')
                driver.execute_script("arguments[0].click();", load_more_button)
                time.sleep(2)  # Wait for the page to load more content
            except (NoSuchElementException, ElementClickInterceptedException):
                break  # If no more "Load More" button is found or click fails, exit loop
    finally:
        driver.quit()  # Make sure to close the driver

    return reviews[:REVIEW_NUMBER]  # Return the list of reviews

In [12]:
# Replace 'tt0120338' with the actual IMDb ID of the movie/show you're interested in
all_titanic_reviews_list = extract_imdb_reviews_all('tt0120338')
all_titanic_reviews_keywords, _ = clean_reviews_english(all_titanic_reviews_list)
print(all_titanic_reviews_keywords)

['watched', 'titanic', 'many', 'times', 'know', 'watch', 'still', 'cry', 'laugh', 'smile', 'feel', 'story', 'flows', 'tension', 'throughout', 'movie', 'two', 'actors', 'acting', 'chemistry', 'need', 'applaud', 'sinking', 'ship', 'realistically', 'filmed', 'heart', 'go', 'perfect', 'fit', 'jack', 'roses', 'love', 'story', 'timeless', 'well', 'movies', 'factors', 'fully', 'qualified', 'say', 'masterpiece']

['dont', 'everytime']
---
['beautiful', 'cinematic', 'movie', 'lots', 'classic', 'extremely', 'sad', 'classic']

['scenesalso', 'timesabsolute', '90s']
---
['record', 'breaker', 'see', 'even', 'day', 'find', 'anything', 'titanic', 'related', 'really', 'interesting', 'great', 'movie']

[]
---
['back', 'remember', 'year', 'bans', 'cloning', 'research', 'unfortunate', 'death', 'princess', 'diana', 'marlins', 'world', 'series', 'woman', 'gave', 'birth', 'also', 'big', 'year', 'release', 'titanic', 'one', 'biggest', 'films', 'time', 'tale', 'ship', 'dreams', 'boy', 'girl', 'fall', 'love', 